<a href="https://colab.research.google.com/github/satuelisa/DBforWeb/blob/main/DBW_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import pandas as pd
url = 'https://raw.githubusercontent.com/satuelisa/DBforWeb/main/example.csv'
csv = pd.read_csv(url)
csv

,Name,Salary,City
0,Gloria,120000,Montreal
1,Ali,86500,Toronto
2,Carlos,96000,Vancouver
3,Marie,78600,Ottawa


So, we have a file in which each row is a *record* and each column is a *field* Let's put that into a *database* with that same structure. In order to respect the structure, we need to know what columns to create in our database.

In [2]:
fields = ','.join(csv.columns)
fields

'Name,Salary,City'

In [3]:
connector = sqlite3.connect('example.db')
cursor = connector.cursor()
cursor.execute(f'CREATE TABLE example ({fields});') # the text we send for execution is an SQL command for the SQLITE3 database system

We now populate the table from the rows in the file. We could of course just create a blank one and then add records to it, but this way we can test stuff out faster. We fill break the dataframe holding the file into a list of records that each is a list of the values of the fields.

In [4]:
rows = csv.to_dict(orient = 'split')['data']
rows

[['Gloria', 120000, 'Montreal'],
 ['Ali', 86500, 'Toronto'],
 ['Carlos', 96000, 'Vancouver'],
 ['Marie', 78600, 'Ottawa']]

Now we are ready to insert them into the table. We need to be able to say how many unknown values we will be providing by specifying a *wildcard string* that matches the field count.

In [5]:
wildcard = ('?,' * len(csv.columns))[:-1]
wildcard

'?,?,?'

In [6]:
cursor.executemany(f'INSERT INTO example ({fields}) VALUES ({wildcard});', rows) # we again send a text to the database server saying what we want done

We can also add more rows in the same way if we like.

In [7]:
more = [['Lin', 50000, 'Quebec'], ['Manu', 76000, 'Calgary']]
cursor.executemany(f'INSERT INTO example ({fields}) VALUES ({wildcard});', more)

Then we need to *commit* the changes we made into the database and then we can disconnect without losing data.

In [8]:
connector.commit()
connector.close()

So, how do we access that database now? We can connect to it again and consult the contents by making *queries*.

In [9]:
connector = sqlite3.connect('example.db')
cursor = connector.execute('SELECT * FROM example;') 
contents = cursor.fetchall()
print(contents)
connector.close()

[('Gloria', 120000, 'Montreal'), ('Ali', 86500, 'Toronto'), ('Carlos', 96000, 'Vancouver'), ('Marie', 78600, 'Ottawa'), ('Lin', 50000, 'Quebec'), ('Manu', 76000, 'Calgary')]


We are now ready to delve into how to define how to group our data into tables, what columns to include in what tables, and then, as a second step, move into how to query more complex aspects about the records that have been stored.